<a href="https://colab.research.google.com/github/soyebganja/Gen-AI-POC-Projects/blob/main/Langchain%20and%20Prompting%20Essentials/Prompt_Templates_%26_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 6.2 MB/s eta 0:00:00


In [2]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException

In [3]:
!pip install python-dotenv

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
prompt = '''
From the below news article, extract revenue and eps in json format containing the
following keys:
- revenue_actual, revenue_expected
- eps_actual, eps_expected

Each value should have a unit such as million, billion cents etc... and currency symbol.

Only return the valid JSON, no preamble.

Article
*********
{article}
'''

pt = ChatPromptTemplate.from_messages([("human", prompt)])
pt

ChatPromptTemplate(input_variables=['article'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['article'], input_types={}, partial_variables={}, template='\nFrom the below news article, extract revenue and eps in json format containing the\nfollowing keys:\n- revenue_actual, revenue_expected\n- eps_actual, eps_expected\n\nEach value should have a unit such as million, billion cents etc... and currency symbol.\n\nOnly return the valid JSON, no preamble.\n\nArticle\n*********\n{article}\n'), additional_kwargs={})])

In [52]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
chain = pt | llm

In [53]:
article_text = '''
Here’s what the company reported compared with what Wall Street was expecting, based on a survey of analysts by LSEG:

Earnings per share: 72 cents, adjusted vs. 58 cents expected
Revenue: $25.18 billion vs. $25.37 billion expected
'''

res = chain.invoke({"article": article_text})
res

AIMessage(content='{"revenue_actual": "$25.18 billion", "revenue_expected": "$25.37 billion", "eps_actual": "72 cents", "eps_expected": "58 cents"}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 156, 'total_tokens': 195, 'completion_time': 0.141818182, 'prompt_time': 0.007713709, 'queue_time': 0.218354012, 'total_time': 0.149531891}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--d5ed9adc-f24d-49bd-85fa-87f6e7938459-0', usage_metadata={'input_tokens': 156, 'output_tokens': 39, 'total_tokens': 195})

In [1]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
chain = pt | llm | JsonOutputParser

NameError: name 'ChatGroq' is not defined

In [58]:
article_text = '''
Here's how the iPhone maker did versus LSEG consensus estimates for the quater ending Sept. 28:

Earnings per share: $1.64, adjusted, versus $1.60 estimated
Revenue: $25.18 billion vs. $25.37 billion expected
'''

res = chain.invoke({"article": article_text})
res.content

'{"revenue_actual": "$25.18 billion", "revenue_expected": "$25.37 billion", "eps_actual": "$1.64", "eps_expected": "$1.60"}'

In [60]:
parser = JsonOutputParser()
try:
    res = parser.parse(res.content)
except OutputParserException as e:
    print(e)

res

{'revenue_actual': '$25.18 billion',
 'revenue_expected': '$25.37 billion',
 'eps_actual': '$1.64',
 'eps_expected': '$1.60'}

In [61]:
type(res)

dict

In [64]:
res["revenue_expected"]

'$25.37 billion'